In [8]:
import os
import numpy as np
import pandas as pd

DATA_PATH = "../data/raw/Dataset.xlsx"
OUT_DIR = "../data/processed"
os.makedirs(OUT_DIR, exist_ok=True)

df = pd.read_excel(DATA_PATH)
print("Original shape:", df.shape)
print("Original exchanges:", df["Sàn"].value_counts(dropna=False).head(10))

# Filter HOSE
df = df[df["Sàn"].isin(["HOSE"])].copy()
df.columns = df.columns.str.strip()

print("\nAfter filtering HOSE:")
print("Shape:", df.shape)
print("Years:", sorted(df["Năm"].dropna().unique().tolist()))
print("Tickers:", df["Mã"].nunique())

Original shape: (7331, 88)
Original exchanges: Sàn
UPCoM    3742
HOSE     1977
HNX      1612
Name: count, dtype: int64

After filtering HOSE:
Shape: (1977, 88)
Years: [2020, 2021, 2022, 2023, 2024]
Tickers: 410


In [9]:
import numpy as np
import pandas as pd

def safe_div(a, b):
    """Chia an toàn: ép numeric, b==0 hoặc NaN -> NaN."""
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce")
    return np.where((b == 0) | (pd.isna(b)), np.nan, a / b)

def find_col(df, keyword):
    """Tìm cột chứa keyword (không phân biệt hoa thường)."""
    kw = keyword.lower()
    matches = [c for c in df.columns if kw in str(c).lower()]
    if matches:
        return matches[0]
    raise KeyError(f"Không tìm thấy cột chứa từ khoá: {keyword}")

In [10]:
# -------------------------
# Map cột (theo Dataset của bạn)
# -------------------------
# Bảng cân đối kế toán
current_assets          = find_col(df, "CĐKT. TÀI SẢN NGẮN HẠN")
current_liabilities     = find_col(df, "CĐKT. Nợ ngắn hạn")
cash                    = find_col(df, "CĐKT. Tiền và tương đương tiền")
short_term_investments  = find_col(df, "CĐKT. Đầu tư tài chính ngắn hạn")
accounts_receivable     = find_col(df, "CĐKT. Các khoản phải thu ngắn hạn")
inventory               = find_col(df, "CĐKT. Hàng tồn kho, ròng")

total_assets            = find_col(df, "CĐKT. TỔNG CỘNG TÀI SẢN")
fixed_assets            = find_col(df, "CĐKT. Tài sản cố định")
total_liabilities       = find_col(df, "CĐKT. NỢ PHẢI TRẢ")
equity                  = find_col(df, "CĐKT. VỐN CHỦ SỞ HỮU")

# KQKD
revenue                 = find_col(df, "KQKD. Doanh thu thuần")
net_income              = find_col(df, "KQKD. Lợi nhuận sau thuế thu nhập doanh nghiệp")
income_tax              = find_col(df, "KQKD. Chi phí thuế thu nhập doanh nghiệp")
interest_expense        = find_col(df, "KQKD. Trong đó: Chi phí lãi vay")

# LCTT / proxy
depreciation            = find_col(df, "LCTT. Khấu hao TSCĐ")
purchase                = find_col(df, "KQKD. Chi phí tài chính")  # proxy theo file của bạn

print("✅ Đã map cột thành công.")

✅ Đã map cột thành công.


In [11]:
# -------------------------
# Chuẩn hoá dữ liệu nền
# -------------------------
keep_cols = [
    "Mã", "Tên công ty", "Sàn",
    "Ngành ICB - cấp 1", "Ngành ICB - cấp 2", "Ngành ICB - cấp 3", "Ngành ICB - cấp 4",
    "Năm",
    current_assets, current_liabilities, cash, short_term_investments, accounts_receivable, inventory,
    total_assets, fixed_assets, total_liabilities, equity,
    revenue, net_income, income_tax, interest_expense,
    depreciation, purchase
]
df_base = df[keep_cols].copy()

id_cols = ["Mã","Tên công ty","Sàn","Ngành ICB - cấp 1","Ngành ICB - cấp 2","Ngành ICB - cấp 3","Ngành ICB - cấp 4","Năm"]
for c in df_base.columns:
    if c not in id_cols:
        df_base[c] = pd.to_numeric(df_base[c], errors="coerce")

df_base = df_base.sort_values(["Mã","Năm"]).reset_index(drop=True)
print("df_base shape:", df_base.shape)

df_base shape: (1977, 24)


In [12]:
# -------------------------
# Tính EBITDA & COGS proxy (giải thích rõ proxy)
# -------------------------
# EBITDA = LNST + Thuế TNDN + Lãi vay + Khấu hao
df_base["EBITDA"] = (
    df_base[net_income]
    + df_base[income_tax]
    + df_base[interest_expense]
    + df_base[depreciation]
)

# Tồn kho đầu kỳ = tồn kho năm trước (theo Mã)
df_base["INV_lag"] = df_base.groupby("Mã")[inventory].shift(1)

# COGS proxy = Tồn kho đầu kỳ + purchase - Tồn kho cuối kỳ
df_base["COGS_proxy"] = df_base["INV_lag"] + df_base[purchase] - df_base[inventory]

df_base[["Mã","Năm","EBITDA","INV_lag","COGS_proxy"]].head(8)

,Mã,Năm,EBITDA,INV_lag,COGS_proxy
0,AAA,2020,333.479711,NaN,NaN
1,AAA,2021,414.687329,946.809380,-250.308920
2,AAA,2022,185.837222,997.384835,-1050.109228
3,AAA,2023,380.986817,1790.091358,769.116786
4,AAA,2024,445.728035,781.680229,-780.050342
5,AAM,2020,-8.846437,NaN,NaN
6,AAM,2021,3.237831,119.394551,34.350380
7,AAM,2022,19.180288,84.836261,-17.261574


In [13]:
# -------------------------
# Tính ratios
# -------------------------
df_ratios = pd.DataFrame({
    "Mã": df_base["Mã"],
    "Tên công ty": df_base["Tên công ty"],
    "Sàn": df_base["Sàn"],
    "Ngành ICB - cấp 1": df_base["Ngành ICB - cấp 1"],
    "Ngành ICB - cấp 2": df_base["Ngành ICB - cấp 2"],
    "Ngành ICB - cấp 3": df_base["Ngành ICB - cấp 3"],
    "Ngành ICB - cấp 4": df_base["Ngành ICB - cấp 4"],
    "Năm": df_base["Năm"],

    # I. Thanh khoản
    "Current_Ratio": safe_div(df_base[current_assets], df_base[current_liabilities]),
    "Quick_Ratio": safe_div(df_base[cash] + df_base[short_term_investments] + df_base[accounts_receivable],
                            df_base[current_liabilities]),
    "Cash_Ratio": safe_div(df_base[cash], df_base[current_liabilities]),

    # II. Đòn bẩy (càng thấp thường càng tốt)
    "Debt_Equity": safe_div(df_base[total_liabilities], df_base[equity]),
    "Net_Leverage": safe_div(df_base[total_liabilities] - df_base[cash], df_base["EBITDA"]),

    # III. Hiệu quả
    "Asset_Turnover": safe_div(df_base[revenue], df_base[total_assets]),
    "Fixed_Asset_Turnover": safe_div(df_base[revenue], df_base[fixed_assets]),
    "Inventory_Turnover": safe_div(df_base["COGS_proxy"], df_base[inventory]),
    "Receivables_Turnover": safe_div(df_base[revenue], df_base[accounts_receivable]),

    # IV. Quy mô
    "ln_Total_Assets": np.log(df_base[total_assets] + 1),
    "ln_Revenue": np.log(df_base[revenue] + 1),

    # V. Sinh lợi
    "ROA": safe_div(df_base[net_income], df_base[total_assets]),
    "ROE": safe_div(df_base[net_income], df_base[equity]),
    "ROS": safe_div(df_base[net_income], df_base[revenue]),
})

df_ratios = df_ratios.drop_duplicates(subset=["Mã","Năm"]).reset_index(drop=True)
print("df_ratios:", df_ratios.shape)
df_ratios.head()

df_ratios: (1977, 22)


/workspaces/FINANCIAL-STATE-TRACKER/venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Mã,Tên công ty,Sàn,Ngành ICB - cấp 1,Ngành ICB - cấp 2,Ngành ICB - cấp 3,Ngành ICB - cấp 4,Năm,Current_Ratio,Quick_Ratio,...,Net_Leverage,Asset_Turnover,Fixed_Asset_Turnover,Inventory_Turnover,Receivables_Turnover,ln_Total_Assets,ln_Revenue,ROA,ROE,ROS
0,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2020,1.191690,0.898923,...,10.740488,0.866869,3.195042,NaN,4.450449,9.056071,8.913222,0.033045,0.070372,0.038119
1,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2021,1.631340,1.285606,...,6.190144,1.313060,6.256122,-0.250965,7.320156,9.211392,9.483729,0.032383,0.059428,0.024663
2,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2022,1.764787,1.142848,...,16.044521,1.416315,7.360197,-0.586623,9.721153,9.287008,9.635039,0.010864,0.019006,0.007671
3,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2023,1.520342,1.262913,...,8.358600,1.089617,6.270061,0.983928,10.475257,9.357419,9.443237,0.026693,0.051844,0.024497
4,AAA,An Phát Bioplastics,HOSE,Nguyên vật liệu,Hóa chất,Hóa chất,"Nhựa, cao su & sợi",2024,1.599783,1.214241,...,11.426707,0.929769,4.208500,-0.576780,7.299066,9.528713,9.455899,0.022558,0.049738,0.024262


In [14]:
out_path = os.path.join(OUT_DIR, "financial_ratios_2020_2024.csv")
df_ratios.to_csv(out_path, index=False, encoding="utf-8-sig")
print("✅ Saved:", out_path)

✅ Saved: ../data/processed/financial_ratios_2020_2024.csv
